# Prepare Workloads

In this notebook, we will try to prepare the workloads that we will be applying
to the deployments. Potentially, this leads to functions that will be called
when we want to send traffic to the deployment and we want to measure the system's
performance.

In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import itertools

import os
import time
import traceback

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

fetching imagenet v2
resizing images


  0%|          | 0/100 [00:00<?, ?it/s]

converting to bentoml files


  0%|          | 0/100 [00:00<?, ?it/s]

extracting base64 files


  0%|          | 0/100 [00:00<?, ?it/s]

preprocessing for mobilenet


  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
config_names = [
    'bentoml-iris-250m-512mb',
    'bentoml-onnx-resnet50-250m-512mb',
    'tfserving-resnetv2-250m-512mb',
    'tfserving-mobilenetv1-250m-512mb',
    'bentoml-pytorch-fashionmnist-250m-512mb',
    'bentoml-keras-toxic-comments-250m-512mb',
]

workload_configs = {}
for exp_config_name in config_names:
    exp_file = f"deployments/{exp_config_name}.json"
    workload_spec = util.load_json_file(exp_file)
    workload_configs[workload_spec['name']] = workload_spec


In [3]:
def experiment_batch(warmup_req_count, total_req_count, batch_size, service_name, cpu_m):
    # apply new cpu configuration to the deployment
    ram_mb = int(cpu_m * 1.5)
    request_limit_override = f"'cpu={cpu_m}m,memory={ram_mb}Mi'"
    print('Request Limit Overrid:', request_limit_override)

    workload_spec = workload_configs[service_name]
    # override request and limit values
    workload_spec['opts']['--request'] = request_limit_override
    workload_spec['opts']['--limit'] = request_limit_override
    kn_command = kube.get_kn_command(**workload_spec)
    print(kn_command)
    !{kn_command}
    print('waiting for settings to converge')
    time.sleep(10)

    # an array as the same length as 
    batch_results = {
        'response_times_ms': [],
    }
    reqs_failed = 0
    reqs_succeeded = 0

    print('warming up...')
    for _ in range(warmup_req_count):
        try:
            # discard the results
            request_funcs.workload_funcs[service_name](batch_size=batch_size)
        except Exception:
            print('exception occured:')
            traceback.print_exc()

    # running the main workload
    print(f'running {service_name} workload function, batch_size: {batch_size}')
    for _ in tqdm(range(total_req_count)):
        try:
            result = request_funcs.workload_funcs[service_name](batch_size=batch_size)
            batch_results['response_times_ms'].append(result['response_time_ms'])
            reqs_succeeded += 1
        except Exception:
            print('exception occured:')
            traceback.print_exc()
            reqs_failed += 1

    batch_results.update({
        'reqs_failed': reqs_failed,
        'reqs_succeeded': reqs_succeeded,
        'ram_mb': ram_mb,
    })

    return batch_results

def perform_experiment_batch(config):
    # configurables = ['batch_size']
    configurables = [k for k in config if isinstance(config[k], list)]
    print(f'configurables: {configurables}')

    # make a copy of config
    config_base = {k:config[k] for k in config if k not in configurables}
    configurable_base = {k:config[k] for k in config if k in configurables}

    results = []
    config_combinations_keys = list(configurable_base.keys())
    for config_combination in itertools.product(*configurable_base.values()):
        configurable = {config_combinations_keys[i]: config_combination[i]  for i in range(len(config_combinations_keys))}
        
        print(configurable)
        new_config = {**config_base}
        new_config.update(configurable)

        result = experiment_batch(**new_config)
        new_config.update(result)
        results.append(new_config)
        
    return results

config = {
    'warmup_req_count': 20,
    'total_req_count': 10,
    'service_name': 'bentoml-iris',
    # cpu_m should always be the first to avoid changing deployment too much
    'cpu_m': [250,1000],
    'batch_size': [1,10,50,100],
}
results = perform_experiment_batch(config)

configurables: ['cpu_m', 'batch_size']
{'cpu_m': 250, 'batch_size': 1}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.064s The Configuration is still working to reflect the latest desired specification.
 10.627s Traffic is not yet migrated to the latest revision.
 10.693s Ingress has not yet been reconciled.
 10.726s Waiting for load balancer to be ready
 10.910s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00174' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/10 [00:00<?, ?it/s]

{'cpu_m': 250, 'batch_size': 10}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-iris'.
Service 'bentoml-iris' with latest revision 'bentoml-iris-00174' (unchanged) is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 10


  0%|          | 0/10 [00:00<?, ?it/s]

{'cpu_m': 250, 'batch_size': 50}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-iris'.
Service 'bentoml-iris' with latest revision 'bentoml-iris-00174' (unchanged) is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 50


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'cpu_m': 1000, 'batch_size': 50}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-iris'.
Service 'bentoml-iris' with latest revision 'bentoml-iris-00175' (unchanged) is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 50


  0%|          | 0/10 [00:00<?, ?it/s]

{'cpu_m': 1000, 'batch_size': 100}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-iris'.
Service 'bentoml-iris' with latest revision 'bentoml-iris-00175' (unchanged) is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 100


  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def post_process(df):
    df['resp_time_avg'] = df.apply(lambda x: np.mean(x['response_times_ms']), axis=1)
    for percentile in [50,90,95,99]:
        df[f'resp_time_p{percentile}'] = df.apply(lambda x: np.percentile(x['response_times_ms'], percentile), axis=1)
    return df


df = pd.DataFrame(data=results)
# df = post_process(df)
df

,warmup_req_count,total_req_count,service_name,cpu_m,batch_size,response_times_ms,reqs_failed,reqs_succeeded,ram_mb
0,20,10,bentoml-iris,250,1,"[50.33, 23.534, 75.613, 90.818, 20.617, 75.118...",0,10,375
1,20,10,bentoml-iris,250,10,"[65.696, 27.116999999999997, 78.261, 95.475000...",0,10,375
2,20,10,bentoml-iris,250,50,"[34.76, 141.656, 94.246, 95.97999999999999, 96...",0,10,375
3,20,10,bentoml-iris,250,100,"[66.72699999999999, 25.096, 87.461, 92.252, 95...",0,10,375
4,20,10,bentoml-iris,1000,1,"[21.22, 19.27, 19.567, 18.443, 19.237000000000...",0,10,1500
5,20,10,bentoml-iris,1000,10,"[20.858, 18.368, 21.253, 19.326, 19.053, 19.29...",0,10,1500
6,20,10,bentoml-iris,1000,50,"[21.465, 23.778000000000002, 21.175, 20.164, 2...",0,10,1500
7,20,10,bentoml-iris,1000,100,"[23.824, 21.898999999999997, 21.75699999999999...",0,10,1500


In [5]:


all_configs = [
    # {
    #     'warmup_req_count': 200,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-iris',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [1000],
    #     'batch_size': [1,5,10,20,50,100],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'tfserving-resnetv2',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [1000],
    #     'batch_size': [1,2,3,5,10],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'tfserving-mobilenetv1',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [1000],
    #     'batch_size': [1,2,3,5],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-onnx-resnet50',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [1000],
    #     'batch_size': [1,2,3,5,10],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-pytorch-fashion-mnist',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [1000],
    #     'batch_size': [1,5,10,20,50,100],
    # },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'bentoml-keras-toxic-comments',
        # cpu_m should always be the first to avoid changing deployment too much
        'cpu_m': [1000],
        'batch_size': [1,5,10,20,50,100],
    },
]
results_folder = './results/batch_experiments_default'


!mkdir -p {results_folder}
for config in all_configs:
    service_name = config['service_name']

    results = perform_experiment_batch(config)
    df = pd.DataFrame(data=results)

    now = util.get_time_with_tz()
    res_name = now.strftime(f'{service_name}_%Y-%m-%d_%H-%M-%S')
    print('res_name:', res_name)
    df.to_csv(f'{results_folder}/{res_name}.csv')

configurables: ['cpu_m', 'batch_size']
{'cpu_m': 1000, 'batch_size': 1}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00004' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 1


  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 1000, 'batch_size': 5}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00004' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 5


  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 1000, 'batch_size': 10}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00004' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 10


...

{'cpu_m': 1000, 'batch_size': 50}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00004' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 50


  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 1000, 'batch_size': 100}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00004' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 100


  0%|          | 0/1000 [00:00<?, ?it/s]

res_name: bentoml-keras-toxic-comments_2021-06-22_17-00-22


In [6]:
all_configs = [
    # {
    #     'warmup_req_count': 200,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-iris',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [250,500,1000,1500,2000],
    #     'batch_size': [1,5,10,20,50,100],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'tfserving-resnetv2',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [500,1000,1500,2000],
    #     'batch_size': [1,2,3,5,10],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'tfserving-mobilenetv1',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [250,500,1000,1500,2000],
    #     'batch_size': [1,2,3,5],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-onnx-resnet50',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [250,500,1000,1500,2000],
    #     'batch_size': [1,2,3,5,10],
    # },
    # {
    #     'warmup_req_count': 20,
    #     'total_req_count': 1000,
    #     'service_name': 'bentoml-pytorch-fashion-mnist',
    #     # cpu_m should always be the first to avoid changing deployment too much
    #     'cpu_m': [250,500,1000,1500,2000],
    #     'batch_size': [1,5,10,20,50,100],
    # },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'bentoml-keras-toxic-comments',
        # cpu_m should always be the first to avoid changing deployment too much
        'cpu_m': [250,500,1000,1500,2000],
        'batch_size': [1,5,10,20,50,100],
    },
]
results_folder = './results/batch_experiments_var_cpu'


!mkdir -p {results_folder}
for config in all_configs:
    service_name = config['service_name']

    results = perform_experiment_batch(config)
    df = pd.DataFrame(data=results)

    now = util.get_time_with_tz()
    res_name = now.strftime(f'{service_name}_%Y-%m-%d_%H-%M-%S')
    print('res_name:', res_name)
    df.to_csv(f'{results_folder}/{res_name}.csv')

configurables: ['cpu_m', 'batch_size']
{'cpu_m': 250, 'batch_size': 1}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-keras-toxic-comments' in namespace 'default':

  0.066s The Configuration is still working to reflect the latest desired specification.
 11.150s Traffic is not yet migrated to the latest revision.
 11.191s Ingress has not yet been reconciled.
 11.297s Waiting for load balancer to be ready
 11.440s Ready to serve.

Service 'bentoml-keras-toxic-comments' applied to latest revision 'bentoml-keras-toxic-comments-00005' is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up

  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 250, 'batch_size': 5}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00005' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 5


  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 250, 'batch_size': 10}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00005' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 10


...

{'cpu_m': 2000, 'batch_size': 50}
Request Limit Overrid: 'cpu=2000m,memory=3000Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=2000m,memory=3000Mi' \
  --request 'cpu=2000m,memory=3000Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00009' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 50


  0%|          | 0/1000 [00:00<?, ?it/s]

{'cpu_m': 2000, 'batch_size': 100}
Request Limit Overrid: 'cpu=2000m,memory=3000Mi'
kn service apply bentoml-keras-toxic-comments --image ghcr.io/nimamahmoudi/bentoml-keras-toxic-comment-classification:20210622155420 \
  --limit 'cpu=2000m,memory=3000Mi' \
  --request 'cpu=2000m,memory=3000Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
No changes to apply to service 'bentoml-keras-toxic-comments'.
Service 'bentoml-keras-toxic-comments' with latest revision 'bentoml-keras-toxic-comments-00009' (unchanged) is available at URL:
http://bentoml-keras-toxic-comments.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-keras-toxic-comments workload function, batch_size: 100


  0%|          | 0/1000 [00:00<?, ?it/s]

res_name: bentoml-keras-toxic-comments_2021-06-22_18-47-01
